### Summarizer

Final BLEU Score: 0.0161
Final ROUGE-1: 0.0288, ROUGE-2: 0.0267, ROUGE-L: 0.0288
Final Perplexity: 45.6756
Final Semantic Similarity: 0.1038

### Custom Training

🔹 Final Evaluation Metrics:
Final BLEU Score: 0.0896
Final ROUGE-1: 0.1318, ROUGE-2: 0.1032, ROUGE-L: 0.1233
Final Perplexity: nan
Final Semantic Similarity: 0.3218

### Code

In [3]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=865d0d97da140d4e93af476fb606a0212f54eaeacccae118180a70afaf64ee7a
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=00a66b4759731b4723fef8c91478d6a8d9c14bacaece251a7214b707e93d4852
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [1]:
import pandas as pd
from collections import defaultdict

df = pd.read_csv("/kaggle/input/final-labels/final_labels.csv") 

conversation_tree = defaultdict(list)
entry_map = {}

for _, row in df.iterrows():
    entry_id = row['entry_id']
    parent_id = row['parent_id']
    link_id = row['link_id']
    text = row['body']
    
    entry_map[entry_id] = text  # Store text
    
    # Identify replies
    if not pd.isna(parent_id):
        conversation_tree[parent_id].append(entry_id)
    elif not pd.isna(link_id):
        conversation_tree[link_id].append(entry_id)

# Identify root posts
root_posts = list(set(entry_map.keys()) - set(conversation_tree.keys()))[:100]


def print_conversation(entry_id, depth=0):
    if entry_id not in entry_map:
        return
    
    prefix = "Post: " if depth == 0 else "Reply: "
    print("  " * depth + f"{prefix}{entry_map[entry_id]}")
    
    for child_id in conversation_tree.get(entry_id, []):
        print_conversation(child_id, depth + 1)

# Print first 100 conversations
for entry_id in root_posts[:50]:
    print_conversation(entry_id)
    print("\n" + "="*50 + "\n") 


Post: My post is still not aporoved. Ive post 10 times already


Post: Oh, fuck off...


Post: nan


Post: You don't even need to go that extreme. There would just be a lot more male prostitutes if the demand for Chads was anywhere as close to the demand men have for women.


Post: Literally her second accident resulting in a death in a decade.

Was high, doing 180km/hr (110mph), looking at her phone whilst disqualified from driving, in a stolen car and evading arrest.

That sentence is so insufficient it is insulting to the victims family.


Post: The Judas release is the way to go, I think. As uncensored as possible until a BR is available.


Post: Plenty of women like or at least don't mind hairy men.


Post: I remember having an existential meltdown once I had life served to me on a silver platter. Multiple girlfriends; I lived to fuck and smoke free weed these hoes were buying me. I was being worshipped like a god.

I had never been so miserable in my life. Shattered it all so I c

In [7]:
import pandas as pd
from transformers import pipeline


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def abstractive_summary(text):
    
    if len(text.split()) < 20:  # Skip very short posts
        return text
    
    summary = summarizer(text, max_length=100, min_length=30, do_sample=False)
    return summary[0]['summary_text']


file_path = "/kaggle/input/final-labels/final_labels.csv" 
df = pd.read_csv(file_path)

# Ensure the correct column exists
if 'body' not in df.columns:
    raise ValueError("Column 'post' not found in dataset. Check CSV structure.")

# Summarize and print each post
for idx, post in enumerate(df["body"].dropna()[:20]):
    print(f"Original Post {idx+1}: {post}\n")
    print(f"Summary {idx+1}: {abstractive_summary(str(post))}\n")
    print("=" * 80)


Device set to use cuda:0
Your max_length is set to 100, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


Original Post 1: Do you have the skin of a 80 year old grandma? Worry no more, just drink water!

Summary 1: Do you have the skin of a 80 year old grandma? Worry no more, just drink water!

Original Post 2: This is taking a grain of truth and extrapolating to insanity.

Stay hydrated, it's healthy, you'll look and feel better. It will not reverse the aging process though.



Your max_length is set to 100, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


Summary 2: Stay hydrated, it's healthy, you'll look and feel better. It will not reverse the aging process though. This is taking a grain of truth and extrapolating to insanity.

Original Post 3: Honestly my favorite thing about this is that they feel the need to cite beauty professionals in order to prove that dehydration is caused by not drinking enough water.



Your max_length is set to 100, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


...

Summary 11: No idea. I watched the movie 10 years or so ago with my dad and all I can remember are the doctors being shocked that the lady was pregnant.

Original Post 12: In the movie there is this group of elderly that discover that they feel suspiciously rejuvenated by going for a bath in a special swimming pool. I was just being silly with the water/de-aging association.



Your max_length is set to 100, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


Summary 12: I was just being silly with the water/de-aging association. In the movie there is this group of elderly that discover that they feel suspiciously rejuvenated by going for a bath in a special swimming pool.

Original Post 13: One of my friends looks like he's frikkin dead. Been trying for years to pour water down his throat and get him to wash his face. Recently won the face washing battle and he looks 5 years younger.



Your max_length is set to 100, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


Summary 13: One of my friends looks like he's frikkin dead. Been trying for years to pour water down his throat and get him to wash his face. Recently won the face washing battle and he looks 5 years younger.

Original Post 14: What kinda Tom Brady nonsense is this

Summary 14: What kinda Tom Brady nonsense is this

Original Post 15: Recommended by F.F

Summary 15: Recommended by F.F

Original Post 16: Professionals say, that dehydration is caused by not drinking enough water. In other news, water is wet, and oranges are orange.

Summary 16: Dehydration is caused by not drinking enough water. Water is wet, and oranges are orange. In other news, water is wet and water is orange.

Original Post 17: Virgina Spread 

Summary 17: Virgina Spread 

Original Post 18: *I can't believe it's not* virgina spread open

Summary 18: *I can't believe it's not* virgina spread open

Original Post 19: I hate you because its the middle of the night and Im silently suffering from laughter but your updoot



In [6]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=494a86012799e253e75de96a955390f5baef4064c7ef557e053da81ffc8f2e1c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [21]:
import pandas as pd
import torch
import numpy as np
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
import math


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


semantic_model = SentenceTransformer("all-MiniLM-L6-v2")


gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2")
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")


def abstractive_summary(text):
    """Generate an abstractive summary using BART"""
    if len(text.split()) < 20:  # Skip very short posts
        return "nan"
    
    summary = summarizer(text, max_length=200, min_length=20, do_sample=False)
    return summary[0]['summary_text']


def calculate_bleu(references, hypotheses):
    """Compute corpus-level BLEU Score"""
    references = [[ref.split()] for ref in references]
    hypotheses = [hyp.split() for hyp in hypotheses]
    return corpus_bleu(references, hypotheses)


def calculate_rouge(references, hypotheses):
    """Compute ROUGE Score for entire dataset"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1, rouge2, rougeL = [], [], []

    for ref, hyp in zip(references, hypotheses):
        if ref.strip() and hyp.strip():
            scores = scorer.score(ref, hyp)
            rouge1.append(scores['rouge1'].fmeasure)
            rouge2.append(scores['rouge2'].fmeasure)
            rougeL.append(scores['rougeL'].fmeasure)

    return np.mean(rouge1), np.mean(rouge2), np.mean(rougeL)


def calculate_perplexity(sentences):
    """Compute Perplexity using GPT-2"""
    text = " ".join(sentences)  # Join all summaries into one text
    tokens_tensor = gpt2_tokenizer.encode(text, return_tensors="pt")

    with torch.no_grad():
        outputs = gpt2_model(tokens_tensor, labels=tokens_tensor)
        loss = outputs.loss.item()
        perplexity = math.exp(loss)
    
    return perplexity


def calculate_semantic_similarity(references, hypotheses):
    """Compute Semantic Similarity"""
    embeddings1 = semantic_model.encode(references, convert_to_tensor=True)
    embeddings2 = semantic_model.encode(hypotheses, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(embeddings1, embeddings2)
    return torch.mean(similarities).item()



file_path = "/kaggle/input/final-labels/final_labels.csv" 
df = pd.read_csv(file_path)

# Ensure correct columns exist
if 'body' not in df.columns or 'highlight' not in df.columns:
    raise ValueError("Columns 'body' and 'highlight' not found in dataset. Check CSV structure.")

# Process all posts
reference_summaries = []
generated_summaries = []
df=df[:50]
for _, row in df.iterrows():
    original_post = str(row["body"])
    reference_summary = str(row["highlight"])

    generated_summary = abstractive_summary(original_post)

    if generated_summary != "nan":
        reference_summaries.append(reference_summary)
        generated_summaries.append(generated_summary)

# Compute aggregated metrics
bleu_score = calculate_bleu(reference_summaries, generated_summaries)
rouge1, rouge2, rougeL = calculate_rouge(reference_summaries, generated_summaries)
perplexity = calculate_perplexity(generated_summaries)
semantic_similarity = calculate_semantic_similarity(reference_summaries, generated_summaries)

# Print final results
print(f"Final BLEU Score: {bleu_score:.4f}")
print(f"Final ROUGE-1: {rouge1:.4f}, ROUGE-2: {rouge2:.4f}, ROUGE-L: {rougeL:.4f}")
print(f"Final Perplexity: {perplexity:.4f}")
print(f"Final Semantic Similarity: {semantic_similarity:.4f}")


Device set to use cuda:0
Your max_length is set to 200, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)
Your max_length is set to 200, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 200, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 200, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Final BLEU Score: 0.0161
Final ROUGE-1: 0.0288, ROUGE-2: 0.0267, ROUGE-L: 0.0288
Final Perplexity: 45.6756
Final Semantic Similarity: 0.1038


## Custom Training

In [22]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments


file_path = "/kaggle/input/final-labels/final_labels.csv"
df = pd.read_csv(file_path)


df = df.dropna(subset=["body", "highlight"])  # Drop if 'body' or 'highlight' is NaN
df = df[df["highlight"].str.strip() != ""]  # Remove rows where 'highlight' is just empty spaces


tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenization Function
def tokenize_function(examples):
    model_inputs = tokenizer(examples["body"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlight"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize_function, batched=True)

# Train-Test Split (80% Train, 20% Test)
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_data = split_dataset["train"]
test_data = split_dataset["test"]  


model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no", 
    save_strategy="epoch",
    per_device_train_batch_size=4,
    num_train_epochs=3,  
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  
)


class CustomTrainer(Trainer):
    def train(self, *args, **kwargs):
        for epoch in range(int(training_args.num_train_epochs)):
            print(f"\n🚀 Training Epoch {epoch + 1}...\n")
            super().train(*args, **kwargs)


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,  
)


trainer.train()

# Save Model
model.save_pretrained("fine_tuned_bart")
tokenizer.save_pretrained("fine_tuned_bart")


Map:   0%|          | 0/752 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(



🚀 Training Epoch 1...



/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,8.167200
20,5.686000
30,2.119300
40,0.543700
50,0.605900
60,0.421900
70,0.207300
80,0.128000
90,0.090300
100,0.197800


...


🚀 Training Epoch 3...



/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,0.019200
20,0.031800
30,0.026600
40,0.019900
50,0.032600
60,0.032600
70,0.023500
80,0.017600
90,0.024500
100,0.025200


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


('fine_tuned_bart/tokenizer_config.json',
 'fine_tuned_bart/special_tokens_map.json',
 'fine_tuned_bart/vocab.json',
 'fine_tuned_bart/merges.txt',
 'fine_tuned_bart/added_tokens.json')

In [26]:
import torch
import numpy as np
import math
from transformers import BartForConditionalGeneration, BartTokenizer
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util

# ✅ Load Fine-Tuned Model from Checkpoint (Step 228)
checkpoint_path = "/kaggle/working/fine_tuned_bart"
model = BartForConditionalGeneration.from_pretrained(checkpoint_path)
tokenizer = BartTokenizer.from_pretrained(checkpoint_path)


semantic_model = SentenceTransformer("all-MiniLM-L6-v2")


from transformers import AutoModelForCausalLM, AutoTokenizer
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2")
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")


def generate_summary(text):
    """Generate summary using fine-tuned BART"""
    input_ids = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=512)
    output = model.generate(input_ids, max_length=128, min_length=30, do_sample=False)
    return tokenizer.decode(output[0], skip_special_tokens=True)


def calculate_bleu(reference, hypothesis):
    """Compute BLEU Score"""
    reference_tokens = [reference.split()]
    hypothesis_tokens = hypothesis.split()
    return sentence_bleu(reference_tokens, hypothesis_tokens)


def calculate_rouge(reference, hypothesis):
    """Compute ROUGE Score"""
    if not reference.strip() or not hypothesis.strip():
        return {"rouge-1": 0, "rouge-2": 0, "rouge-l": 0}

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return {
        "rouge-1": scores["rouge1"].fmeasure,
        "rouge-2": scores["rouge2"].fmeasure,
        "rouge-l": scores["rougeL"].fmeasure
    }


def calculate_perplexity(sentence):
    """Compute Perplexity using GPT-2"""
    if not sentence.strip(): 
        return float("inf")  # Assign a high perplexity for empty outputs

    tokens_tensor = gpt2_tokenizer.encode(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = gpt2_model(tokens_tensor, labels=tokens_tensor)
        loss = outputs.loss.item()
        perplexity = math.exp(loss)

    return perplexity



def calculate_semantic_similarity(reference, hypothesis):
    """Compute Semantic Similarity"""
    ref_embedding = semantic_model.encode(reference, convert_to_tensor=True)
    hyp_embedding = semantic_model.encode(hypothesis, convert_to_tensor=True)
    return util.pytorch_cos_sim(ref_embedding, hyp_embedding).item()


# Initialize accumulators for final scores
total_bleu = 0
total_rouge_1 = 0
total_rouge_2 = 0
total_rouge_l = 0
total_perplexity = 0
total_similarity = 0
num_samples = 0

# Evaluate on test set
for idx, row in test_data.to_pandas().iterrows():
    original_post = str(row["body"])
    reference_summary = str(row["highlight"])

    generated_summary = generate_summary(original_post)

    if not generated_summary.strip():  
        print(f"⚠️ Skipping empty summary at index {idx}")
        continue  # ✅ Skip empty summaries

    # Compute metrics
    bleu_score = calculate_bleu(reference_summary, generated_summary)
    rouge_scores = calculate_rouge(reference_summary, generated_summary)
    perplexity = calculate_perplexity(generated_summary)
    semantic_similarity = calculate_semantic_similarity(reference_summary, generated_summary)

    # Accumulate scores
    total_bleu += bleu_score
    total_rouge_1 += rouge_scores["rouge-1"]
    total_rouge_2 += rouge_scores["rouge-2"]
    total_rouge_l += rouge_scores["rouge-l"]
    total_perplexity += perplexity
    total_similarity += semantic_similarity
    num_samples += 1


# Compute final averages
final_bleu = total_bleu / num_samples
final_rouge_1 = total_rouge_1 / num_samples
final_rouge_2 = total_rouge_2 / num_samples
final_rouge_l = total_rouge_l / num_samples
final_perplexity = total_perplexity / num_samples
final_similarity = total_similarity / num_samples

# Print final scores
print("\n🔹 Final Evaluation Metrics:")
print(f"Final BLEU Score: {final_bleu:.4f}")
print(f"Final ROUGE-1: {final_rouge_1:.4f}, ROUGE-2: {final_rouge_2:.4f}, ROUGE-L: {final_rouge_l:.4f}")
print(f"Final Perplexity: {final_perplexity:.4f}")
print(f"Final Semantic Similarity: {final_similarity:.4f}")


⚠️ Skipping empty summary at index 0
⚠️ Skipping empty summary at index 1
⚠️ Skipping empty summary at index 2
⚠️ Skipping empty summary at index 3
⚠️ Skipping empty summary at index 4
⚠️ Skipping empty summary at index 5
⚠️ Skipping empty summary at index 6
⚠️ Skipping empty summary at index 7
⚠️ Skipping empty summary at index 8
⚠️ Skipping empty summary at index 9
⚠️ Skipping empty summary at index 10
⚠️ Skipping empty summary at index 11
⚠️ Skipping empty summary at index 12
⚠️ Skipping empty summary at index 13
⚠️ Skipping empty summary at index 14
⚠️ Skipping empty summary at index 15
⚠️ Skipping empty summary at index 16
⚠️ Skipping empty summary at index 17
⚠️ Skipping empty summary at index 18
⚠️ Skipping empty summary at index 19
⚠️ Skipping empty summary at index 20


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

⚠️ Skipping empty summary at index 22
⚠️ Skipping empty summary at index 23
⚠️ Skipping empty summary at index 24
⚠️ Skipping empty summary at index 25
⚠️ Skipping empty summary at index 26
⚠️ Skipping empty summary at index 27
⚠️ Skipping empty summary at index 28
⚠️ Skipping empty summary at index 29
⚠️ Skipping empty summary at index 30
⚠️ Skipping empty summary at index 31
⚠️ Skipping empty summary at index 32
⚠️ Skipping empty summary at index 33
⚠️ Skipping empty summary at index 34
⚠️ Skipping empty summary at index 35
⚠️ Skipping empty summary at index 36
⚠️ Skipping empty summary at index 37
⚠️ Skipping empty summary at index 38
⚠️ Skipping empty summary at index 39
⚠️ Skipping empty summary at index 40
⚠️ Skipping empty summary at index 41
⚠️ Skipping empty summary at index 42
⚠️ Skipping empty summary at index 43
⚠️ Skipping empty summary at index 44
⚠️ Skipping empty summary at index 45
⚠️ Skipping empty summary at index 46
⚠️ Skipping empty summary at index 47
⚠️ Skipping 

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

⚠️ Skipping empty summary at index 126
⚠️ Skipping empty summary at index 127
⚠️ Skipping empty summary at index 128
⚠️ Skipping empty summary at index 129
⚠️ Skipping empty summary at index 130
⚠️ Skipping empty summary at index 131


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

⚠️ Skipping empty summary at index 133
⚠️ Skipping empty summary at index 134
⚠️ Skipping empty summary at index 135
⚠️ Skipping empty summary at index 136
⚠️ Skipping empty summary at index 137
⚠️ Skipping empty summary at index 138
⚠️ Skipping empty summary at index 139
⚠️ Skipping empty summary at index 140
⚠️ Skipping empty summary at index 141
⚠️ Skipping empty summary at index 142
⚠️ Skipping empty summary at index 143
⚠️ Skipping empty summary at index 144
⚠️ Skipping empty summary at index 145
⚠️ Skipping empty summary at index 146
⚠️ Skipping empty summary at index 147
⚠️ Skipping empty summary at index 148
⚠️ Skipping empty summary at index 149
⚠️ Skipping empty summary at index 150

🔹 Final Evaluation Metrics:
Final BLEU Score: 0.0896
Final ROUGE-1: 0.1318, ROUGE-2: 0.1032, ROUGE-L: 0.1233
Final Perplexity: nan
Final Semantic Similarity: 0.3218
